# Script saldo Cajamar, OpenBank y Unicaja

## Requisites

In [ ]:
!pip install plotly

## Code

### Imports

In [ ]:
import plotly as py
import plotly.graph_objs as go

import numpy as np
import datetime as dt
import os.path
import xlrd

py.offline.init_notebook_mode(connected=True)

### Read CSV

In [ ]:
def processCSV(files, splitToken, bank):
    fecha = []
    saldo = []
    
    if bank == 'cajamar':
        for file in files:
            first = True
            if os.path.exists(file):
                with open(file) as f:
                    for line in f:
                        if first is False:
                            tokens = line.split(splitToken)
                            fecha.append(dt.datetime.strptime(tokens[0], '%d/%m/%Y').date())
                            saldo.append(float(tokens[len(tokens)-1][:-1].replace('.', '').replace(',', '.')))
                        first = False

    if bank == 'openbank':
        for file in files:
            first = 0
            if os.path.exists(file):
                with open(file) as f:
                    for line in f:                        
                        if first > 11:
                            tokens = line.split(splitToken)
                            fecha.append(dt.datetime.strptime(tokens[1], '%d/%m/%Y').date())
                            saldo.append(float(tokens[len(tokens)-1][:-1].replace('.', '').replace(',', '.')))
                        first += 1

    fecha = np.array(fecha)
    saldo = np.array(saldo)

    return (saldo, fecha)

### Process data files

#### Cajamar y Openbank

In [ ]:
files = ['saldo.csv']
(saldosCajamar, fechasCajamar) = processCSV(files, '\t', 'cajamar')

files = ['Nomina.csv']
(saldosOpenNomina, fechasOpenNomina) = processCSV(files, '\t', 'openbank')

files = ['Ahorro.csv']
(saldosOpenAhorro, fechasOpenAhorro) = processCSV(files, '\t', 'openbank')

saldosOpenAhorro = np.insert(saldosOpenAhorro, 0, saldosOpenAhorro[0], axis=0)
fechasOpenAhorro = np.insert(fechasOpenAhorro, 0, fechasOpenNomina[0], axis=0)

### Traces

#### Cajamar y Openbank

In [ ]:
trace_main_cajamar = go.Scatter(
    x = fechasCajamar,
    y = saldosCajamar,
    name = 'Cajamar: Saldo ' + str(format(saldosCajamar[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'green'
    ),
    mode = 'lines'
)
trace_max_cajamar = go.Scatter(
    x = fechasCajamar,
    y = [saldosCajamar.max() for f in fechasCajamar],
    name = 'Cajamar: Saldo máximo',
    #visible = 'legendonly',
    #hoverinfo = 'name',
    line = dict(
        color = 'cyan',
        width = 4,
        dash = 'dot'
    )
)
trace_mean_cajamar = go.Scatter(
    x = fechasCajamar,
    y = [saldosCajamar.mean() for f in fechasCajamar],
    name = 'Cajamar: Saldo medio',
    #hoverinfo = 'none',
    line = dict(
        color = 'magenta',
        width = 4,
        dash = 'dashdot'
    )
)
trace_min_cajamar = go.Scatter(
    x = fechasCajamar,
    y = [saldosCajamar.min() for f in fechasCajamar],
    name = 'Cajamar: Saldo mínimo',
    line = dict(
        color = 'red',
        width = 4,
        dash = 'dot'
    )
)

trace_main_openAhorro = go.Scatter(
    x = fechasOpenAhorro,
    y = saldosOpenAhorro,
    name = 'OpenAhorro: Saldo ' + str(format(saldosOpenAhorro[0], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'blue'
    ),
    mode = 'lines'
)

trace_main_openNomina = go.Scatter(
    x = fechasOpenNomina,
    y = saldosOpenNomina,
    name = 'OpenNomina: Saldo ' + str(format(saldosOpenNomina[0], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'darkred'
    ),
    mode = 'lines'
)

data = [trace_main_cajamar, trace_max_cajamar, trace_mean_cajamar, trace_min_cajamar,
        trace_main_openAhorro, trace_main_openNomina]

### Plot

#### Cajamar y Openbank

In [ ]:
layout = go.Layout(title = 'Saldo ' + str(fechasCaja[-1]) + ' / ' + str(fechasOpen[0]),
              xaxis = dict(title = 'Fecha'),
              yaxis = dict(title = 'Saldo (€)'),
              showlegend = True
)

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='styled-line')

### Full Unicaja Auto

In [ ]:
def processXLS(file, bank):   
    balance = []
    date = []
    if bank == 'unicaja':
        book = xlrd.open_workbook(file)
        first_sheet = book.sheet_by_index(0)
        date_int = first_sheet.col_values(0)[5:]
        balance = first_sheet.col_values(5)[5:]
        date = [xlrd.xldate_as_datetime(date_int[i], book.datemode).date().isoformat() for i in  xrange(0, len(date_int))]
    return (balance, date)

print("Files found:")
path = "accounts/current/"
directories = [f for f in os.listdir(path) if os.path.isdir(path+f)]
bankdata = {}
for bankname in directories:
    bankdata[bankname] = {"fechas":[], "saldos":[]}
    files = [os.path.join(path+bankname,f) for f in os.listdir(path+bankname)]
    files.sort(key=lambda x: os.path.getmtime(x))
    for datafile in files:
        extension = os.path.splitext(datafile)[1]
        if extension == ".xls":
            print("> "+bankname+"/"+datafile)
            (balance, date) = processXLS(datafile, bankname)
            bankdata[bankname]["saldos"].extend(balance)
            bankdata[bankname]["fechas"].extend(date)

saldos = bankdata["unicaja"]["saldos"]
fechas = bankdata["unicaja"]["fechas"]

trace_main_unicaja = go.Scatter(
    x = fechas,
    y = saldos,
    name = 'Unicaja: Saldo ' + str(format(saldos[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'green'
    ),
    mode = 'lines'
)

data = [trace_main_unicaja]

layout = go.Layout(title = 'Saldo ' + str(fechas[-1]),
              xaxis = dict(title = 'Fecha'),
              yaxis = dict(title = 'Saldo (€)'),
              showlegend = True
)
fig = dict(data=data, layout=layout)
py.offline.iplot(fig)
